# Web Scrape Amazon Page Notebook
I will be web scraping the data that will go into a MongoDB database.

In [1]:
#Imports the BeautifulSoup module from library bs4
import requests
from bs4 import BeautifulSoup

# Webscrape Part 1: Obtain Individual Item Links

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
amazon_url_tvs = 'https://www.amazon.com/s/ref=lp_1266092011_nr_n_12?fst=as%3Aoff&rh=n%3A172282%2Cn%3A%21493964%2Cn%3A1266092011%2Cn%3A172659&bbn=1266092011&ie=UTF8&qid=1564525104&rnid=1266092011'


In [3]:
#Access the first Amazon Page
page = requests.get(amazon_url_tvs, headers=headers, timeout=5) #it will keep trying at this stage, the loop will not continue to iterate until a response is given
c = page.content
amazon_url_tvs_page_contents = BeautifulSoup(c, 'html.parser')


In [4]:
amazon_url_tvs_page_contents


<!DOCTYPE doctype html>
<html class="a-no-js" data-19ax5a9jf="dingo"><head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/><!--  emit CSM JS -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){if(1==window.ueinit)try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);


    var ue_err_chan = 'jserr-rw';
(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{

In [17]:
len(amazon_url_tvs_page_contents.find_all("ul"))

7